In [1]:
from keras.models import Model
from keras import layers
from keras import initializers

#hyper-parameters
image_height = image_width = 224
image_channels = 3
cardinality = 32

def AddCommonLayers(x):
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.)(x)
    return x

def ResidualBlockGenerator(x, channels_in, channels_out, stepChanger=False):
    #stepChanger is for reducing size of the feature map like 56 x 56 to 28 x 28
    if stepChanger:
        shortcut = x
        group_size = (channels_in)//(cardinality)
        groups =[]
        for i in range(cardinality):
            groupsElements = layers.Conv2D(group_size, kernel_size=(1,1), strides=(2,2), padding='same')(x)
            groupsElements = AddCommonLayers(groupsElements)
            groupsElements = layers.Conv2D(group_size, kernel_size=(3,3), padding='same')(groupsElements)
            groupsElements = AddCommonLayers(groupsElements)
            groups.append(groupsElements)
        x = layers.concatenate(groups)
        x = layers.Conv2D(channels_out, kernel_size=(1,1))(x)
        x = layers.BatchNormalization()(x)

        layer = layers.Conv2D(channels_out, kernel_size=(2,2), strides=(2,2), use_bias=False, kernel_initializer=initializers.Ones())
        layer.trainable = False
        shortcut = layer(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
        x = layers.add([shortcut, x])
        x = layers.LeakyReLU(alpha=0.)(x)
    else:
        shortcut = x
        group_size = (channels_in)//(cardinality)
        groups=[]
        for i in range(cardinality):
            groupsElements = layers.Conv2D(group_size, kernel_size=(1,1))(x)
            groupsElements = AddCommonLayers(groupsElements)
            groupsElements = layers.Conv2D(group_size, kernel_size=(3,3), padding='same')(groupsElements)
            groupsElements = AddCommonLayers(groupsElements)
            groups.append(groupsElements)
        x = layers.concatenate(groups)
        x = layers.Conv2D(channels_out, kernel_size=(1,1))(x)
        x = layers.BatchNormalization()(x)

        if shortcut.shape[3] != x.shape[3]:
            layer = layers.Conv2D(channels_out, kernel_size=(1,1), use_bias=False, kernel_initializer = initializers.Ones())
            layer.trainable = False
            shortcut = layer(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        x = layers.add([shortcut, x])
        x = layers.LeakyReLU(alpha=0.)(x)
    return x

def NetworkBuilder(_input):
    x = layers.Conv2D(64, kernel_size=(7,7), strides=(2,2), padding='same')(_input)
    x = AddCommonLayers(x)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

    # Residual Blocks Section
    for i in range(3):
        x = ResidualBlockGenerator(x, 128, 256, stepChanger=False) # Any number should work

    for i in range(4):
        if i==0:
            _stepChanger = True
        else:
            _stepChanger = False
        x = ResidualBlockGenerator(x, 256, 512, stepChanger = _stepChanger)

    for i in range(6):
        if i==0:
            _stepChanger = True
        else:
            _stepChanger = False
        x = ResidualBlockGenerator(x, 512, 1024, stepChanger = _stepChanger)

    for i in range(3):
        if i == 0:
            _stepChanger = True
        else:
            _stepChanger = False
        x = ResidualBlockGenerator(x, 1024, 2048, stepChanger = _stepChanger)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1000, activation='softmax')(x)

    return x

input_tensor = layers.Input(shape=(image_height, image_width, image_channels))
output_tensor = NetworkBuilder(input_tensor)

model = Model(inputs=[input_tensor], outputs = [output_tensor])
print(model.summary())

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

__________________________________________________________________________________________________
leaky_re_lu_237 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_238[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_239 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_240[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_241 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_242[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_243 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_244[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_245 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_246[0][0]    
__________

__________________________________________________________________________________________________
leaky_re_lu_436 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_438[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_438 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_440[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_440 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_442[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_442 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_444[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_444 (LeakyReLU)     (None, 28, 28, 8)    0           batch_normalization_446[0][0]    
__________

batch_normalization_597 (BatchN (None, 14, 14, 16)   64          conv2d_597[0][0]                 
__________________________________________________________________________________________________
batch_normalization_599 (BatchN (None, 14, 14, 16)   64          conv2d_599[0][0]                 
__________________________________________________________________________________________________
batch_normalization_601 (BatchN (None, 14, 14, 16)   64          conv2d_601[0][0]                 
__________________________________________________________________________________________________
batch_normalization_603 (BatchN (None, 14, 14, 16)   64          conv2d_603[0][0]                 
__________________________________________________________________________________________________
batch_normalization_605 (BatchN (None, 14, 14, 16)   64          conv2d_605[0][0]                 
__________________________________________________________________________________________________
batch_norm

                                                                 batch_normalization_849[0][0]    
__________________________________________________________________________________________________
leaky_re_lu_846 (LeakyReLU)     (None, 14, 14, 1024) 0           add_13[0][0]                     
__________________________________________________________________________________________________
conv2d_850 (Conv2D)             (None, 7, 7, 32)     32800       leaky_re_lu_846[0][0]            
__________________________________________________________________________________________________
conv2d_852 (Conv2D)             (None, 7, 7, 32)     32800       leaky_re_lu_846[0][0]            
__________________________________________________________________________________________________
conv2d_854 (Conv2D)             (None, 7, 7, 32)     32800       leaky_re_lu_846[0][0]            
__________________________________________________________________________________________________
conv2d_856

Total params: 33,384,936
Trainable params: 22,290,280
Non-trainable params: 11,094,656
__________________________________________________________________________________________________
None
